In [1]:
import duckdb  # Importa o DuckDB para manipulação de dados e execução de SQL
import os  # Importa o módulo os para interagir com variáveis de ambiente do sistema
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env
from deltalake import DeltaTable, write_deltalake

# Carrega as variáveis de ambiente definidas no arquivo .env
load_dotenv(find_dotenv())

# Define as credenciais de acesso ao MinIO a partir das variáveis de ambiente
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")  # Chave de acesso do MinIO
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Chave secreta do MinIO
HOST_MINIO = os.getenv("HOST_MINIO")  # Host do MinIO

# Conecta ao DuckDB, criando uma instância de conexão
con = duckdb.connect()

# Cria uma secret no DuckDB para acessar o MinIO com as credenciais fornecidas
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

# Define o caminho de destino no MinIO para os arquivos Parquet
path_minio_landing = 's3://landing/comex'
path_minio_bronze = 's3://bronze/comex/ncm'

In [3]:
storage_options = {
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:9000",
    "AWS_REGION": "us-east-1",
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_KEY,
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
    "AWS_ALLOW_HTTP": "true"
}

In [4]:
df = con.sql(f"""
    SELECT 
        CO_NCM AS cod_ncm,
        CO_UNID AS cod_unid,
        CO_SH6 AS cod_sh6,
        CO_PPE AS cod_ppe,
        CO_PPI AS cod_ppi,
        CO_FAT_AGREG AS cod_fat_agreg,
        CO_CUCI_ITEM AS cod_cuci_item,
        CO_CGCE_N3 AS cod_cgce_n3,
        CO_SIIT AS cod_siit,
        CO_ISIC_CLASSE AS cod_isic_classe,
        CO_EXP_SUBSET AS cod_exp_subset,
        NO_NCM_POR AS no_ncm_por
    FROM '{path_minio_landing}/NCM.parquet';
        """).to_arrow_table()

In [5]:
df.to_pandas().head()

,cod_ncm,cod_unid,cod_sh6,cod_ppe,cod_ppi,cod_fat_agreg,cod_cuci_item,cod_cgce_n3,cod_siit,cod_isic_classe,cod_exp_subset,no_ncm_por
0,60063130,10,600631,3853.0,3853.0,3.0,65529,240,4000.0,1391,1699.0,"Outros tecidos de malha, de fibras sintéticas,..."
1,60063190,10,600631,3853.0,3853.0,3.0,65529,240,4000.0,1391,1699.0,"Outros tecidos de malha, de fibras sintéticas,..."
2,60063200,10,600632,3853.0,3853.0,3.0,65529,240,4000.0,1391,1699.0,"Outros tecidos de malha, de fibras sintéticas,..."
3,60063210,10,600632,3853.0,3853.0,3.0,65529,240,4000.0,1391,1699.0,"Outros tecidos de malha, de fibras sintéticas,..."
4,60063220,10,600632,3853.0,3853.0,3.0,65529,240,4000.0,1391,1699.0,"Outros tecidos de malha, de fibras sintéticas,..."


In [10]:
write_deltalake(
        f'{path_minio_bronze}',
        df,
        storage_options=storage_options
    )

In [6]:
table_path = f'{path_minio_bronze}'

# Conecte à tabela Delta existente
table = DeltaTable(table_path, storage_options=storage_options)

In [7]:
table.merge(
    source=df,
    predicate='target.cod_ncm = source.cod_ncm',
    source_alias="source",
    target_alias="target",
).when_not_matched_insert_all().execute()

{'num_source_rows': 13699,
 'num_target_rows_inserted': 11075,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 11075,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 249,
 'scan_time_ms': 0,
 'rewrite_time_ms': 177}

In [8]:
con.sql(f"""
        SELECT *
        FROM delta_scan('{path_minio_bronze}')
        """)

┌──────────┬──────────┬─────────┬─────────┬───┬──────────┬─────────────────┬────────────────┬──────────────────────┐
│ cod_ncm  │ cod_unid │ cod_sh6 │ cod_ppe │ … │ cod_siit │ cod_isic_classe │ cod_exp_subset │      no_ncm_por      │
│ varchar  │  int64   │ varchar │ varchar │   │ varchar  │     varchar     │    varchar     │       varchar        │
├──────────┼──────────┼─────────┼─────────┼───┼──────────┼─────────────────┼────────────────┼──────────────────────┤
│ 60063220 │       10 │ 600632  │ 3853.0  │ … │ 4000.0   │ 1391            │ 1699.0         │ Outros tecidos de …  │
│ 60063230 │       10 │ 600632  │ 3853.0  │ … │ 4000.0   │ 1391            │ 1699.0         │ Outros tecidos de …  │
│ 57023200 │       15 │ 570232  │ 3990.0  │ … │ 4000.0   │ 1393            │ 1699.0         │ Tapete, etc, de ma…  │
│ 57025100 │       15 │ 570251  │ 3990.0  │ … │ 4000.0   │ 1393            │ 1699.0         │ Tapete, etc, de lã…  │
│ 57025200 │       15 │ 570252  │ 3990.0  │ … │ 4000.0   │ 1393 

In [9]:
con.close()